In [2]:
import json
from collections import Counter
from functools import reduce
import numpy as np
import re
from os import listdir
from os import close
import codecs 
import mediawiki
from joblib import Parallel, delayed

wikipedia = mediawiki.MediaWiki(lang=u'ru')

In [ ]:
path = 'wiki_extracted/'
alf = listdir(path)

def walker(a):
    categories = []
    page_ids = []
    articles = []
    _path = path + a
    __path = [_path + '/' + p for p in listdir(_path)]
    print(a, end=' ')
    for p in __path:
        print(p[18:], end=' ')
        with codecs.open(p, 'r', 'utf-8-sig') as f:
            file = f.read()
            file = re.sub(r'\n+', '. ', file)
            file = re.split(r'</doc>', file)
            for f in file[:-1]:
                id = re.findall(r'id="(.*)" url', f)[0]
                try:
                    categories.extend(wikipedia.page(pageid=int(id)).categories)
                    page_ids.append(int(id))
                    articles.append(re.findall(r'">\. (.*)', f)[0])
                except KeyboardInterrupt:
                    close()
                except:
                    print("Page with id " + str(id) + " not found")
    
    return categories, page_ids, articles

categories, page_ids, articles = Parallel(n_jobs=4)(delayed(walker)(a) for a in alf)

In [ ]:
#with open('output.json') as f:
    #info = json.load(f)
    #print(len(info))
    #categories = [i["categories"] for i in info]
    #categories = reduce(lambda x,y:x+y, categories)
categories = [c[10:] for c in categories]
print(sorted(set([c for c in categories if re.findall(r'.* /d/d/d/d .*', c)])))
counts = Counter(categories)
print(np.mean(list(counts.values())))
print(counts)